In [2]:
# 1-) Yeni Urunlerin Sektorleri.xlsx

import pandas as pd

def load_and_process_new_products():
    # Kullanacağımız yılları tanımlıyoruz
    years = [2002, 2007, 2012, 2017, 2022]
    file_paths = {
        2002: r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS02_V202401\product_codes_HS02_V202401.csv",
        2007: r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS07_V202401\product_codes_HS07_V202401.csv",
        2012: r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS12_V202401\product_codes_HS12_V202401.csv",
        2017: r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS17_V202401\product_codes_HS17_V202401.csv",
        2022: r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS22_V202401\product_codes_HS22_V202401.csv"
    }
    
    # Her yıl için ürün kodlarını ekliyoruz
    dfs = {year: pd.read_csv(file_paths[year]) for year in years}
    product_sets = {year: set(dfs[year]['code']) for year in years}

    # Yeni ürünleri buluyoruz
    new_products_dfs = []
    for i in range(1, len(years)):
        year = years[i]
        prev_year = years[i - 1]
        new_products = product_sets[year] - product_sets[prev_year]
        
        # Yıl kolonunu ekliyoruz
        new_df = dfs[year][dfs[year]['code'].isin(new_products)].copy()
        new_df['Yıl'] = year
        new_products_dfs.append(new_df)
    
    # Bütün yıllardaki yeni ürünleri tek dataframede topluyoruz
    combined_new_products = pd.concat(new_products_dfs, ignore_index=True)
    
    # 2 basamaklı HS kodunu hesaplıyoruz
    combined_new_products['2 Basamaklı HS Kodu'] = combined_new_products['code'] // 10000
    
    # Sektör kodlarını ve isimilerini kullanarak birleştiriyoruz
    atlas_df = pd.read_excel('0-) HS92 Sektor Kodlari ve Isimleri.xlsx')
    atlas_df['hs2'] = atlas_df['hs2'].astype(str).str.zfill(2)
    combined_new_products['2 Basamaklı HS Kodu'] = combined_new_products['2 Basamaklı HS Kodu'].astype(str).str.zfill(2)
    combined_new_products = combined_new_products.merge(
        atlas_df[['hs2', 'hs2_name', 'sector']],
        left_on='2 Basamaklı HS Kodu',
        right_on='hs2',
        how='left'
    ).rename(columns={
        'code': '6 Basamaklı HS Kodu',
        'hs2_name': '2 Basamaklı HS İsmi',
        'sector': 'Sektör İsmi'
    }).drop(columns=['hs2'])

    # Yıl sütununu en başa alıyoruz
    cols = ['Yıl'] + [col for col in combined_new_products.columns if col != 'Yıl']
    combined_new_products = combined_new_products[cols]
    return combined_new_products

# Fonksiyonu uyguluyoruz
processed_new_products = load_and_process_new_products()

# Sadece Sektör İsmi 'Textiles' olanları filtreliyoruz
textiles_only = processed_new_products[processed_new_products['Sektör İsmi'] == 'Textiles ']

# "4 Basamaklı HS Kodu" sütunu ekliyoruz
textiles_only['4 Basamaklı HS Kodu'] = textiles_only['6 Basamaklı HS Kodu'].astype(str).str[:4]

# Kaydediyoruz
textiles_only.to_excel("1-) Yeni Urunlerin Sektorleri.xlsx", index=False)

C:\Users\oguzo\AppData\Local\Temp\ipykernel_13368\158877336.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  textiles_only['4 Basamaklı HS Kodu'] = textiles_only['6 Basamaklı HS Kodu'].astype(str).str[:4]


In [3]:
# Toplu datayı birleştirme tekrar çağırarak vakit kaybetmemek için

# Yıllar, HS versiyonları ve dosya yolu formatı
yillar = list(range(2012, 2022))
hs_versiyonlari = {year: 'HS12' if year <= 2016 else 'HS17' for year in yillar}
dosya_yolu_format = r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_{hs_version}_V202401\BACI_{hs_version}_Y{year}_V202401.csv"

# Atlas dosyasını oku ve sütunları temizle
combined_atlas_df = pd.read_excel('0-) HS92 Sektor Kodlari ve Isimleri.xlsx')
combined_atlas_df['sector'] = combined_atlas_df['sector'].str.strip()
combined_atlas_df['hs2_name'] = combined_atlas_df['hs2_name'].str.strip()

# Boş bir liste başlangıçta tutuyoruz
all_data = []

# Her yılın verisini işle ve listeye ekle
for year in yillar:
    hs_version = hs_versiyonlari[year]
    dosya_yolu = dosya_yolu_format.format(hs_version=hs_version, year=year)
    print(f"İşleniyor: {dosya_yolu}")
    df = pd.read_csv(dosya_yolu)
    
    # HS2 ve HS4 kodunu hesapla ve sektörel bilgilerle birleştir
    df['hs2'] = df['k'] // 10000  
    df = df.merge(combined_atlas_df, on='hs2', how='left')  
    
    # Yıla göre birleştirilen veriyi işaretleyelim
    df['year'] = year
    
    # Listeye ekle
    all_data.append(df)

# Tüm yılları birleştir
merged_df = pd.concat(all_data, ignore_index=True)

# Elde edilen veri çerçevesini kontrol et
merged_textiles_df = merged_df[merged_df["sector"] == "Textiles"]
merged_textiles_df["hs4"] = merged_textiles_df["k"] // 100
merged_df["hs4"] = merged_df["k"] // 100

İşleniyor: C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS12_V202401\BACI_HS12_Y2012_V202401.csv
İşleniyor: C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS12_V202401\BACI_HS12_Y2013_V202401.csv
İşleniyor: C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS12_V202401\BACI_HS12_Y2014_V202401.csv
İşleniyor: C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS12_V202401\BACI_HS12_Y2015_V202401.csv
İşleniyor: C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS12_V202401\BACI_HS12_Y2016_V202401.csv
İşleniyor: C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS17_V202401\BACI_HS17_Y2017_V202401.csv
İşleniyor: C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS17_V202401\BACI_HS17_Y2018_V202401.csv
İşleniyor: C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS17_V202401\BACI_HS17_Y2019_V202401.csv
İşleniyor: C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS17_V202401\BACI_HS17_Y2020_V202

C:\Users\oguzo\AppData\Local\Temp\ipykernel_13368\1206094459.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_textiles_df["hs4"] = merged_textiles_df["k"] // 100


In [7]:
# HS2 textiles listesini belirliyoruz

# Çağırıp boşlukları temizliyoruz
combined_atlas_df = pd.read_excel('0-) HS92 Sektor Kodlari ve Isimleri.xlsx')
combined_atlas_df['sector'] = combined_atlas_df['sector'].str.strip()
textiles_hs2 = combined_atlas_df[combined_atlas_df['sector'] == 'Textiles'][['hs2', 'hs2_name']]

# Kodları bir listeye koyuyoruz
textiles_hs2_codes = textiles_hs2['hs2'].dropna().unique().tolist()

# İsimleriyle beraber bir sözlük oluşturuyoruz
textiles_hs2_dict = textiles_hs2.set_index('hs2')['hs2_name'].str.strip().to_dict()

In [16]:
# 2-) Turkiye'de tekstil ticaret hacmi 

# Türkiye kodu
turkey_code = 792

# Türkiye'nin dahil olduğu tüm ticaret
turkiye_ticaret = merged_df[
    (merged_df['i'] == turkey_code) | (merged_df['j'] == turkey_code)
]

# Türkiye'nin tekstil ticareti
turkiye_textiles = merged_df[
    (merged_df['sector'] == 'Textiles') & 
    ((merged_df['i'] == turkey_code) | (merged_df['j'] == turkey_code))
]

# Yıllara göre toplam ticaret hacmini hesaplıyoruz
toplam_ticaret_hacmi = turkiye_ticaret.groupby('year')['v'].sum().reset_index()
textiles_ticaret_hacmi = turkiye_textiles.groupby('year')['v'].sum().reset_index()

# Tekstil ticaretinin toplam ticarete oranını hesaplıyoruz
oran_df = toplam_ticaret_hacmi.copy()
oran_df['Türkiyenin Toplam Tekstil Ticaret Hacmi'] = textiles_ticaret_hacmi['v']
oran_df['Tekstil Oranı (%)'] = (oran_df['Türkiyenin Toplam Tekstil Ticaret Hacmi'] / oran_df['v']) * 100
oran_df.rename(columns={"v": "Türkiyenin Toplam Ticaret Hacmi"}, inplace=True)
oran_df.rename(columns={"year": "Yıl"}, inplace=True)
pd.options.display.float_format = '{:,.2f}'.format

# Excel dosyasına kaydediyoruz
oran_df.to_excel("2-) Turkiye'de tekstil ticaret hacmi.xlsx", index=False)

In [11]:
# 3 HS2 Oranı

# Türkiye kodu
turkey_code = 792

# İlgili HS2 kodlarını listele
combined_atlas_df = pd.read_excel('0-) HS92 Sektor Kodlari ve Isimleri.xlsx')

# Türkiye'nin dahil olduğu tüm ticareti filtrele
turkiye_ticaret = merged_df[
    (merged_df['i'] == turkey_code) | (merged_df['j'] == turkey_code)
]

# Yıllara göre toplam ticaret hacmini hesapla
toplam_ticaret_hacmi = turkiye_ticaret.groupby('year')['v'].sum().reset_index()
toplam_ticaret_hacmi.rename(columns={'v': 'toplam_hacim'}, inplace=True)

# Boş bir liste oluştur
hs2_oran_listesi = []

# Her HS2 kodu için ayrı ayrı hesaplama yap
for hs2 in textiles_hs2_codes:
    # Belirli HS2 kodunu filtrele
    turkiye_hs2 = turkiye_ticaret[turkiye_ticaret['hs2'] == hs2]
    
    # HS2 kodu için ticaret hacmini hesapla
    hs2_ticaret_hacmi = turkiye_hs2.groupby('year')['v'].sum().reset_index()
    hs2_ticaret_hacmi.rename(columns={'v': f'hs2_{hs2}_hacim'}, inplace=True)
    hs2_ticaret_hacmi['hs2'] = hs2  # HS2 kodunu ekle
    
    # Toplam ticaret hacmiyle birleştir ve oranı hesapla
    hs2_ticaret_hacmi = hs2_ticaret_hacmi.merge(toplam_ticaret_hacmi, on='year', how='left')
    hs2_ticaret_hacmi[f'hs2_{hs2}_orani'] = (hs2_ticaret_hacmi[f'hs2_{hs2}_hacim'] / hs2_ticaret_hacmi['toplam_hacim']) * 100
    
    # Boş değerleri doldur
    hs2_ticaret_hacmi.fillna(0, inplace=True)
    
    # Listeye ekle
    hs2_oran_listesi.append(hs2_ticaret_hacmi[['year', 'hs2', f'hs2_{hs2}_hacim', f'hs2_{hs2}_orani']])

# Tüm HS2 verilerini birleştir
final_df = pd.concat(hs2_oran_listesi, ignore_index=True)

# Sonuçları görüntüle
pd.options.display.float_format = '{:,.2f}'.format
final_df.to_excel("3.xlsx", index= False)


In [89]:
import pandas as pd

# Dosyayı yükle
file_path = '0-) HS92 Sektor Kodlari ve Isimleri.xlsx'
combined_atlas_df = pd.read_excel(file_path)

# "sector" sütunundaki boşlukları temizle
combined_atlas_df['sector'] = combined_atlas_df['sector'].str.strip()

# "Textiles" sektörüne ait HS2 kodlarını ve isimlerini al
textiles_hs2 = combined_atlas_df[combined_atlas_df['sector'] == 'Textiles'][['hs2', 'hs2_name']]

# Kodları ayrı bir listeye al
textiles_hs2_codes = textiles_hs2['hs2'].dropna().unique().tolist()

# Kodların isimlerini bir sözlükte tut
textiles_hs2_dict = textiles_hs2.set_index('hs2')['hs2_name'].str.strip().to_dict()

# Sonuçları yazdır
print("HS2 Kodları Listesi:", textiles_hs2_codes)
print("HS2 Kodları ve İsimleri Sözlüğü:", textiles_hs2_dict)


HS2 Kodları Listesi: [50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 94, 97]
HS2 Kodları ve İsimleri Sözlüğü: {50: 'Silk', 51: 'Wool', 52: 'Cotton', 53: 'Other vegetable textile fibres', 54: 'Man-made filaments', 55: 'Man-made staple fibres', 56: 'Wadding, felt and nonwovens', 57: 'Carpets', 58: 'Special woven fabrics', 59: 'Impregnated, coated or laminated textile fabrics', 60: 'Knitted fabrics', 61: 'Apparel, knit', 62: 'Apparel, not knit', 63: 'Other made up textile articles', 64: 'Footwear', 65: 'Headgear', 66: 'Umbrellas and walking-sticks', 67: 'Feathers and down', 94: 'Furniture', 97: 'Art'}


In [93]:
# 4 Oranı ama türkiye tekstiline

import pandas as pd

# Türkiye kodu
turkey_code = 792

# HS2 kodlarını içeren veri dosyasını yükle
combined_atlas_df = pd.read_excel('0-) HS92 Sektor Kodlari ve Isimleri.xlsx')

# İlgili HS2 kodlarını listele
hs2_list = list(range(50, 68)) + [94, 97]

# Türkiye'nin dahil olduğu tüm ticareti filtrele
turkiye_ticaret = merged_df[
    (merged_df['i'] == turkey_code) | (merged_df['j'] == turkey_code)
]

# Türkiye'nin tekstil sektöründeki ticaretini filtrele
turkiye_textiles = turkiye_ticaret[turkiye_ticaret['hs2'].isin(hs2_list)]

# Türkiye'nin toplam tekstil ticaret hacmini yıl bazında hesapla
toplam_textile_hacmi = turkiye_textiles.groupby('year')['v'].sum().reset_index()ß
toplam_textile_hacmi.rename(columns={'v': 'toplam_textile_hacim'}, inplace=True)

# Boş bir liste oluştur
hs2_oran_listesi = []

# Her HS2 kodu için ayrı ayrı hesaplama yap
for hs2 in hs2_list:
    # Belirli HS2 kodunu filtrele
    turkiye_hs2 = turkiye_textiles[turkiye_textiles['hs2'] == hs2]
    
    # HS2 kodu için ticaret hacmini hesapla
    hs2_ticaret_hacmi = turkiye_hs2.groupby('year')['v'].sum().reset_index()
    hs2_ticaret_hacmi.rename(columns={'v': f'hs2_{hs2}_hacim'}, inplace=True)
    hs2_ticaret_hacmi['hs2'] = hs2  # HS2 kodunu ekle
    
    # Toplam tekstil hacmiyle birleştir ve oranı hesapla
    hs2_ticaret_hacmi = hs2_ticaret_hacmi.merge(toplam_textile_hacmi, on='year', how='left')
    hs2_ticaret_hacmi[f'hs2_{hs2}_orani'] = (hs2_ticaret_hacmi[f'hs2_{hs2}_hacim'] / hs2_ticaret_hacmi['toplam_textile_hacim']) * 100
    
    # Boş değerleri doldur
    hs2_ticaret_hacmi.fillna(0, inplace=True)
    
    # Listeye ekle
    hs2_oran_listesi.append(hs2_ticaret_hacmi[['year', 'hs2', f'hs2_{hs2}_hacim', f'hs2_{hs2}_orani']])

# Tüm HS2 verilerini birleştir
final_df = pd.concat(hs2_oran_listesi, ignore_index=True)

# Sonuçları Excel'e kaydet
final_df.to_excel("4.xlsx", index=False)

SyntaxError: invalid syntax (1008195525.py, line 36)

In [88]:
final_df

,year,hs2,hs2_50_hacim,hs2_50_orani,hs2_51_hacim,hs2_51_orani,hs2_52_hacim,hs2_52_orani,hs2_53_hacim,hs2_53_orani,...,hs2_65_hacim,hs2_65_orani,hs2_66_hacim,hs2_66_orani,hs2_67_hacim,hs2_67_orani,hs2_94_hacim,hs2_94_orani,hs2_97_hacim,hs2_97_orani
0,2012,50,"40,907.17",0.09,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013,50,"51,666.33",0.10,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,50,"56,346.76",0.11,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,50,"48,073.16",0.10,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016,50,"40,435.25",0.09,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2017,97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"84,590.30",0.18
196,2018,97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"63,630.94",0.13
197,2019,97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"260,673.73",0.55
198,2020,97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"30,995.48",0.07


In [57]:
country_codes = r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS17_V202401\country_codes_V202401.csv"
a = pd.read_csv(country_codes)
a

,country_code,country_name,country_iso2,country_iso3
0,4,Afghanistan,AF,AFG
1,8,Albania,AL,ALB
2,12,Algeria,DZ,DZA
3,16,American Samoa,AS,ASM
4,20,Andorra,AD,AND
...,...,...,...,...
233,876,Wallis and Futuna Isds,WF,WLF
234,882,Samoa,WS,WSM
235,887,Yemen,YE,YEM
236,891,Serbia and Montenegro (...2005),CS,SCG


In [94]:
# İlk yaptığım oranlarda hata var
import pandas as pd

# Dosya yolları
country_codes_file = r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS17_V202401\country_codes_V202401.csv"

# Ülke kodları dosyasını yükle
country_codes_df = pd.read_csv(country_codes_file)

# Türkiye'nin tekstil sektöründeki ticareti filtrele
turkey_code = 792
hs2_list = list(range(50, 68)) + [94, 97]  # Tekstil HS2 kodları

# Türkiye'nin dahil olduğu tekstil ticaretini filtrele
turkiye_textiles = merged_df[
    (merged_df['hs2'].isin(hs2_list)) & 
    ((merged_df['i'] == turkey_code) | (merged_df['j'] == turkey_code))
]

# Türkiye'nin yıllık toplam tekstil ticaret hacmini hesapla
toplam_textile_hacmi = turkiye_textiles.groupby('year')['v'].sum().reset_index()
toplam_textile_hacmi.rename(columns={'v': 'toplam_textile_hacim'}, inplace=True)

# İhracat yapan ülkeleri filtrele (i = Türkiye, j = Diğer ülke)
ihracat = turkiye_textiles[turkiye_textiles['i'] == turkey_code]
ihracat_hacmi = ihracat.groupby(['year', 'j'])['v'].sum().reset_index()
ihracat_hacmi.rename(columns={'j': 'country_code', 'v': 'export_value'}, inplace=True)

# İthalat yapan ülkeleri filtrele (j = Türkiye, i = Diğer ülke)
ithalat = turkiye_textiles[turkiye_textiles['j'] == turkey_code]
ithalat_hacmi = ithalat.groupby(['year', 'i'])['v'].sum().reset_index()
ithalat_hacmi.rename(columns={'i': 'country_code', 'v': 'import_value'}, inplace=True)

# Ülke isimlerini ekle (ihracat)
ihracat_hacmi = ihracat_hacmi.merge(country_codes_df, left_on='country_code', right_on='country_code', how='left')
ihracat_hacmi = ihracat_hacmi.merge(toplam_textile_hacmi, on='year', how='left')
ihracat_hacmi['export_ratio'] = (ihracat_hacmi['export_value'] / ihracat_hacmi['toplam_textile_hacim']) * 100

# Ülke isimlerini ekle (ithalat)
ithalat_hacmi = ithalat_hacmi.merge(country_codes_df, left_on='country_code', right_on='country_code', how='left')
ithalat_hacmi = ithalat_hacmi.merge(toplam_textile_hacmi, on='year', how='left')
ithalat_hacmi['import_ratio'] = (ithalat_hacmi['import_value'] / ithalat_hacmi['toplam_textile_hacim']) * 100

# İhracat ve ithalatı ayrı ayrı Excel dosyasına kaydet
ihracat_hacmi[['year', 'country_name', 'export_value', 'export_ratio']].to_excel("turkiye_textiles_export.xlsx", index=False)
ithalat_hacmi[['year', 'country_name', 'import_value', 'import_ratio']].to_excel("turkiye_textiles_import.xlsx", index=False)

print("İhracat ve ithalat analizleri tamamlandı, Excel dosyalarına kaydedildi!")


İhracat ve ithalat analizleri tamamlandı, Excel dosyalarına kaydedildi!


In [96]:
import pandas as pd

# Dosya yolları
country_codes_file = r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_HS17_V202401\country_codes_V202401.csv"

# Ülke kodları dosyasını yükle
country_codes_df = pd.read_csv(country_codes_file)

# Türkiye'nin tekstil sektöründeki ticareti filtrele
turkey_code = 792
hs2_list = list(range(50, 68)) + [94, 97]  # Tekstil HS2 kodları

# Türkiye'nin dahil olduğu tekstil ticaretini filtrele
turkiye_textiles = merged_df[
    (merged_df['hs2'].isin(hs2_list)) & 
    ((merged_df['i'] == turkey_code) | (merged_df['j'] == turkey_code))
]

# Türkiye'nin toplam tekstil ihracat ve ithalat hacmini yıl bazında ayrı ayrı hesapla
toplam_textile_ihracat = turkiye_textiles[turkiye_textiles['i'] == turkey_code].groupby('year')['v'].sum().reset_index()
toplam_textile_ihracat.rename(columns={'v': 'toplam_textile_ihracat'}, inplace=True)

toplam_textile_ithalat = turkiye_textiles[turkiye_textiles['j'] == turkey_code].groupby('year')['v'].sum().reset_index()
toplam_textile_ithalat.rename(columns={'v': 'toplam_textile_ithalat'}, inplace=True)

# İhracat yapan ülkeleri filtrele (i = Türkiye, j = Diğer ülke)
ihracat = turkiye_textiles[turkiye_textiles['i'] == turkey_code]
ihracat_hacmi = ihracat.groupby(['year', 'j'])['v'].sum().reset_index()
ihracat_hacmi.rename(columns={'j': 'country_code', 'v': 'export_value'}, inplace=True)

# İthalat yapan ülkeleri filtrele (j = Türkiye, i = Diğer ülke)
ithalat = turkiye_textiles[turkiye_textiles['j'] == turkey_code]
ithalat_hacmi = ithalat.groupby(['year', 'i'])['v'].sum().reset_index()
ithalat_hacmi.rename(columns={'i': 'country_code', 'v': 'import_value'}, inplace=True)

# Ülke isimlerini ekle (ihracat)
ihracat_hacmi = ihracat_hacmi.merge(country_codes_df, left_on='country_code', right_on='country_code', how='left')
ihracat_hacmi = ihracat_hacmi.merge(toplam_textile_ihracat, on='year', how='left')
ihracat_hacmi['export_ratio'] = (ihracat_hacmi['export_value'] / ihracat_hacmi['toplam_textile_ihracat']) * 100

# Ülke isimlerini ekle (ithalat)
ithalat_hacmi = ithalat_hacmi.merge(country_codes_df, left_on='country_code', right_on='country_code', how='left')
ithalat_hacmi = ithalat_hacmi.merge(toplam_textile_ithalat, on='year', how='left')
ithalat_hacmi['import_ratio'] = (ithalat_hacmi['import_value'] / ithalat_hacmi['toplam_textile_ithalat']) * 100

# İhracat ve ithalatı ayrı ayrı Excel dosyasına kaydet
ihracat_hacmi[['year', 'country_name', 'export_value', 'export_ratio']].to_excel("turkiye_textiles_export_ratio_corrected.xlsx", index=False)
ithalat_hacmi[['year', 'country_name', 'import_value', 'import_ratio']].to_excel("turkiye_textiles_import_ratio_corrected.xlsx", index=False)

print("Düzeltilmiş analizler tamamlandı, Excel dosyalarına kaydedildi!")


Düzeltilmiş analizler tamamlandı, Excel dosyalarına kaydedildi!


In [ ]:
# RCA Hesabı yapan kod
# Her seferinde vakit kaybetmeyeyim diye dış ticaret verilerini bir data frame'e kaydediyorum

import pandas as pd

# Yıllar ve HS versiyonları
yillar = list(range(2012, 2022))
hs_versiyonlari = {year: 'HS12' if year <= 2016 else 'HS17' for year in yillar}

# Dosya yolu formatı
dosya_yolu_format = r"C:\Users\oguzo\OneDrive - Fintegral\Masaüstü\MENA\BACI\BACI_{hs_version}_V202401\BACI_{hs_version}_Y{year}_V202401.csv"
# Tüm yılları içeren DataFrame'i birleştir
combined_atlas_df = pd.read_excel('0-) HS92 Sektor Kodlari ve Isimleri.xlsx')

# Ülkeleri yükle
ulkeler_df = pd.read_excel('0-) Ulkeler.xlsx')  
combined_atlas_df['sector'] = [zz.strip() for zz in combined_atlas_df['sector']] 
combined_atlas_df['hs2_name'] = [zz.strip() for zz in combined_atlas_df['hs2_name']] 

# Her yılın verisini oku ve birleştir
for year in [yillar[0]]:
    hs_version = hs_versiyonlari[year] 
    dosya_yolu = dosya_yolu_format.format(hs_version=hs_version, year=year)  
    df = pd.read_csv(dosya_yolu)
    df['hs2'] = df['k']//10000
    df = df.merge(combined_atlas_df)

    ### RCA HS6
    x_p = df[['t','i', 'k','v']].groupby(['t','i','k']).sum().reset_index()
    x_p.columns = ['t','i','k','prod_country']

    x_c = df[['t','i', 'v']].groupby(['t','i']).sum().reset_index()
    x_c.columns = ['t','i','country']

    w_p = df[['t', 'k','v']].groupby(['t','k']).sum().reset_index()
    w_p.columns = ['t','k','prod']

    w = df['v'].sum()

    rca = x_p.merge(x_c).merge(w_p)
    rca['w'] = w
    rca['RCA_HS6'] = (rca['prod_country']/rca['country'])/(rca['prod']/rca['w'])

    df = df.merge(rca[['t','i','k','RCA_HS6']])

    ### RCA HS2

    x_p = df[['t','i', 'hs2','v']].groupby(['t','i','hs2']).sum().reset_index()
    x_p.columns = ['t','i','hs2','prod_country']

    x_c = df[['t','i', 'v']].groupby(['t','i']).sum().reset_index()
    x_c.columns = ['t','i','country']

    w_p = df[['t', 'hs2','v']].groupby(['t','hs2']).sum().reset_index()
    w_p.columns = ['t','hs2','prod']

    w = df['v'].sum()

    rca = x_p.merge(x_c).merge(w_p)
    rca['w'] = w
    rca['RCA_HS2'] = (rca['prod_country']/rca['country'])/(rca['prod']/rca['w'])

    df = df.merge(rca[['t','i','hs2','RCA_HS2']])


    ### RCA HS4

    x_p = df[['t','i', 'sector','v']].groupby(['t','i','sector']).sum().reset_index()
    x_p.columns = ['t','i','sector','prod_country']

    x_c = df[['t','i', 'v']].groupby(['t','i']).sum().reset_index()
    x_c.columns = ['t','i','country']

    w_p = df[['t', 'sector','v']].groupby(['t','sector']).sum().reset_index()
    w_p.columns = ['t','sector','prod']

    w = df['v'].sum()

    rca = x_p.merge(x_c).merge(w_p)
    rca['w'] = w
    rca['RCA_sector'] = (rca['prod_country']/rca['country'])/(rca['prod']/rca['w'])

    df = df.merge(rca[['t','i','sector','RCA_sector']])

    df = df[df['i'].isin(ulkeler_df['country_code'])]